In [3]:
import pandas as pd
import os
from skimage.transform import resize
from sklearn.utils.validation import column_or_1d
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor

scorer = make_scorer(mean_squared_error, greater_is_better=False)
param_grid={'C':[0.001],'gamma':[0.1],'kernel':['rbf']}
svr=svm.SVR(max_iter=200)
model = svr
#model = SGDRegressor()
#model=GridSearchCV(svr, param_grid, scoring = scorer)

flat_data_arr=[] #input array
target_arr=[] #output array

#file path
path = 'C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data'
surInfoFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/survival_info.csv"
MetaDataFile = "C:/Users/a0981/OneDrive/桌面/archive/BraTS2020_training_data/content/data/meta_data.csv"

date_column = ['Brats20ID','Age','Survival_days','Extent_of_Resection']
df_surInfo = pd.read_csv(surInfoFile,names= date_column,skiprows=1)
df_metaData = pd.read_csv(MetaDataFile,names= ['slice_path','target','volume','slice'],skiprows=1)


def modify_value(value):
    idStr = str(value)
    number_str = idStr[-3:] 
    number_int = int(number_str)
    return number_int

#for index,row in df_surInfo.iterrows():
 #   idStr = str(row['Brats20ID'])
  #  number_str = idStr[-3:] 
  #  number_int = int(number_str)
   # surInfoId.append(number_int)
    
df_surInfo['Brats20ID'] = df_surInfo['Brats20ID'].apply(modify_value)
print(df_surInfo)
print(df_metaData)






     Brats20ID     Age  Survival_days Extent_of_Resection
0            1  60.463            289                 GTR
1            2  52.263            616                 GTR
2            3  54.301            464                 GTR
3            4  39.068            788                 GTR
4            5  68.493            465                 GTR
..         ...     ...            ...                 ...
230        363  57.000             62                 GTR
231        366  72.000            633                 GTR
232        367  60.000            437                 STR
233        368  49.000            442                 GTR
234        369  50.000            145                 GTR

[235 rows x 4 columns]
                                  slice_path  target  volume  slice
0         /content/data/volume_41_slice_0.h5       0      41      0
1         /content/data/volume_41_slice_1.h5       0      41      1
2         /content/data/volume_41_slice_2.h5       0      41      2
3       

In [4]:
pattern = r"volume_(\d+)_"
pattern2 = r"slice_(\d+)"

y = pd.DataFrame(columns = ['Survival_days','Volume'])
volumes = []
#path which contains all the categories of images
fileCount = 0
volumeCount = 0
volumeCountMax = 20

volumeId = -1
MSE = 0.0
dataCount = 0
for img in os.listdir(path):
    
    if img.endswith(".h5"):
        
        fileName = os.path.join(path,img)
        dataId = re.search(pattern, fileName).group(1)
        dataId = int(dataId)
        SliceId = re.search(pattern2, fileName).group(1)
        SliceId = int(SliceId)
        
        if dataId in df_surInfo['Brats20ID'].values:
            
            
            if volumeId != dataId :
                volumeId = dataId
                df_volume = df_metaData.loc[df_metaData['volume'] == volumeId, ['slice','target']]
                
                volumeCount += 1
                #print(df_volume)
            
            target = df_volume.loc[df_volume['slice'] == SliceId ,'target'].iloc[0]
            
            if volumeCount == volumeCountMax + 1:
            
                flat_data=np.array(flat_data_arr)
                df = pd.DataFrame(flat_data)

                #print(df)
                print(volumes)
                x=df.iloc[:,:-2] #input data 
                Y = df.iloc[:,-2]

                #print(x)
                #print(Y.ravel())
                
                num = int(volumeCountMax*0.78)
                train_volume = volumes[:num]
                test_volume = volumes[num:]
                print(train_volume)
                print(test_volume)
                
                df_train = df[df[df.columns[-1]].isin(train_volume)]
                #print(df_train)
                df_test = df[df[df.columns[-1]].isin(test_volume)]
                #print(df_test)
                
                x_train = df_train.iloc[:,:-2]
                y_train = df_train.iloc[:,-2]
                x_test = df_test.iloc[:,:-2]
                y_test = df_test.iloc[:,-2]
                        
                #x_train,x_test,y_train,y_test=train_test_split(x,Y,test_size=0.20,random_state=77)
                print('Splitted Successfully')
                print(x_train.shape)
                print(y_train.shape)
                print(x_test.shape)
                print(y_test.shape)
                #model.partial_fit(x_train,y_train)
                model.fit(x_train,y_train)
                print('The Model is trained well with the given images')

                y_pred=model.predict(x_test)
                print("The predicted Data is :")
                print(y_pred)
                print("The actual data is:")
                print(np.array(y_test))
                mse = mean_squared_error(y_pred,y_test)
                MSE += mse*len(y_test)
                dataCount += len(y_test)
                print(f"The model MSE is {mse}")

                flat_data_arr = []
                y = pd.DataFrame(columns = ['Survival_days'])
                fileCount = 0
                volumeCount = 1
                volumes = []
            
            if target == 1:
                
                #print(f'dataId :{dataId}')
                #print(f'SliceId :{SliceId}')
                f = h5py.File(fileName, 'r')
                img_array = f.get('image')
                img_array = np.array(img_array)
                img_array = img_array.flatten()
                
                if dataId not in volumes:
                    volumes.append(dataId)
            
                age = df_surInfo[df_surInfo['Brats20ID']==dataId]['Age'].values[0]
                img_array = np.append(img_array,age)
                survDays = df_surInfo.loc[df_surInfo['Brats20ID'] == dataId, 'Survival_days'].iloc[0]
                img_array = np.append(img_array,survDays)
                img_array = np.append(img_array,dataId)
            
                flat_data_arr.append(img_array)
                fileCount += 1
            
print(f"dataCount is {dataCount}")
print(f"The MSE is {MSE/dataCount}")    


[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 10, 110, 111, 112, 113, 114, 115, 116, 117, 118]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 10, 110, 111, 112, 113]
[114, 115, 116, 117, 118]
Splitted Successfully
(933, 230401)
(933,)
(345, 230401)
(345,)


C:\Users\a0981\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


The Model is trained well with the given images
The predicted Data is :
[324.0870689  324.52677672 324.90015547 325.09580367 325.25625324
 325.32659828 325.31122271 325.2307979  325.09205303 324.87354241
 324.62696871 324.34716521 324.090343   323.94937712 323.83894299
 323.79293647 323.80439441 323.81510409 323.79352599 323.77330745
 323.82675886 323.83723253 323.89646764 323.96552679 324.05794813
 324.10078995 324.18827938 324.31216445 324.37585771 324.28729474
 324.05476297 323.9716205  327.63955347 327.96640687 328.22999019
 328.34698212 326.21145332 326.29471566 326.21133816 326.01451865
 325.71374836 325.5516554  325.43314522 325.31509753 325.25994356
 325.1696103  325.1593462  325.16434206 325.06544204 324.99672037
 324.90534841 324.67348873 324.42727444 324.21088014 323.95889119
 323.65138443 323.35305216 323.13402908 322.89975292 322.64910444
 322.33042153 322.11310444 321.90563794 321.68575619 321.48809832
 321.37399645 321.22585726 321.09023619 320.95057456 320.85358885
 320

C:\Users\a0981\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


The Model is trained well with the given images
The predicted Data is :
[504.58700242 505.22167504 505.80984639 506.34063042 507.04699952
 507.7903795  508.52111977 509.27432457 509.85633412 510.29607002
 510.73229259 511.05675768 511.4477744  511.69398882 511.89423246
 512.01751238 511.94890607 511.97881823 511.9494925  498.37631802
 497.8848896  497.39217098 496.88171255 496.57275816 496.50676245
 496.50215706 496.60335974 496.85650057 497.26661423 497.68332374
 498.05303163 498.39546019 498.53960101 498.65854133 498.92265476
 499.21478816 499.50289403 499.80059965 500.00422871 500.20356985
 500.35373927 500.30012058 500.19808656 500.14524404 500.00396555
 499.92407837 499.90888118 500.04011101 500.08867184 500.08043531
 500.04965358 500.11444479 500.16145509 500.01726613 499.9529805
 499.92370612 499.91843222 499.91286294 499.79681148 499.82503673
 499.81308838 499.75389035 499.77052654 499.87250126 499.97030687
 500.08685222 500.18102495 500.37681057 500.65997407 500.88721194
 501.

...

The Model is trained well with the given images
The predicted Data is :
[260.7745575  260.70342457 260.67841681 260.68238325 260.64700053
 260.57651555 260.58982362 260.58636375 260.61966368 260.59832811
 260.51021381 260.48034873 260.59102161 260.72701955 260.9052309
 261.1420438  261.31046074 261.48186703 261.7040499  261.93511291
 262.04918285 262.16534749 262.24077986 262.33552163 262.39268226
 262.45971353 262.70840837 262.95794617 261.83419811 261.90141614
 262.04872846 262.11219142 262.08081259 262.10140718 262.12805283
 262.15382563 262.13652473 262.24290365 262.36910339 262.39412276
 262.3060057  262.25213065 262.30507429 262.30837816 262.29813922
 262.3134918  262.32674806 262.3382407  262.33727284 262.34110466
 262.33872158 262.35148725 262.38860749 262.35668072 262.33590531
 262.34052625 262.31337547 262.32085113 262.26502015 262.1911083
 262.14159356 262.15688953 262.16391347 262.25085133 262.30841931
 262.27038677 262.29943846 262.35406822 262.39374559 262.32728441
 262.1

C:\Users\a0981\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


The Model is trained well with the given images
The predicted Data is :
[397.14465482 396.85545582 404.90426039 404.76316198 404.77716134
 404.78492704 404.93518389 405.28393585 405.53483959 405.92921806
 406.34121421 406.79350064 407.18179656 407.40190829 407.5791157
 407.64177545 407.83502127 408.00278991 408.02934885 408.0339165
 407.97735937 407.89757525 407.77642049 407.44432273 406.99988954
 406.56520327 406.02280986 405.46514734 405.00065056 404.49327547
 404.08347652 403.63533105 403.18336374 402.72080224 402.30776061
 401.9771135  401.45661564 401.03766582 400.56368902 400.01926668
 399.55081252 399.11003103 398.70979513 398.39100396 397.95924993
 397.55468984 396.17754228 395.77885788 395.39474742 395.11053623
 395.09139412 395.22022656 395.39290512 395.59792805 395.81899686
 396.12034    396.39203245 396.66664453 396.93858547 397.16072492
 397.37701784 397.62483079 397.92887397 398.18742319 398.46203503
 398.60610328 398.7252784  398.30677123 398.21555302 398.05962804
 397.8

C:\Users\a0981\anaconda3\envs\sklearn-env\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


The Model is trained well with the given images
The predicted Data is :
[230.80668981 230.73145668 230.67620172 230.6141227  230.52391021
 230.33068408 230.111526   230.02238193 229.90292172 229.78786994
 229.63163851 229.44671498 229.24654764 229.1067204  228.86501861
 228.64413548 228.46543429 228.28106562 228.22376432 228.31070005
 228.41319612 228.3292943  230.40873844 230.423075   230.43575322
 230.48413136 230.52264898 230.57260323 230.69420677 230.71866184
 230.68814791 230.66269821 230.70151542 230.71298315 230.73019238
 230.85914381 231.05889701 231.17716956 231.29105262 231.42061859
 231.5054984  231.48169212 231.39658617 231.36980844 231.40617933
 231.43743224 231.43807696 231.34768832 231.27695935 231.09063392
 230.90208675 230.84026851 230.87191506 226.7953952  227.04078129
 227.2996727  227.46750563 227.60966824 227.76062475 227.88690605
 228.03399164 228.09823028 228.01762617 228.05617597 228.06306513
 228.23397825 228.47842655 228.57233098 228.60683302 228.66789274
 228